In [1]:
import tensorflow as tf
import numpy as np
import threading
import time as ti

In [2]:
# --- 분석 모델로 분석 수행 및 결과 저장 클래스 ---
class EsvibAnalyzer:
    def __init__(self, modeldir):
        self.modeldir = modeldir
        self.elapsedTime = 0
        self.predictList = []
        
    def analysis(self, inputList):
        with tf.Session(graph=tf.Graph()) as sess:
            print("START : " + self.modeldir)
            
            # 모델 로딩
            tf.saved_model.loader.load(sess, ["ver1"], self.modeldir)
            
            graph = tf.get_default_graph()
            x = graph.get_tensor_by_name("in:0")
            use_dropout = graph.get_tensor_by_name("use_dropout:0")
            out = graph.get_tensor_by_name("out:0")
            
            # 모델 분석 수행
            startTime = ti.time()
            _out = sess.run([out], feed_dict={x:inputList, use_dropout:False})
            endTime = ti.time()
            
            self.elapsedTime = endTime - startTime
            
            # 분석 결과 저장
            for index in range(0, inputList.shape[0]):
                self.predictList.append(np.argmax(_out[0][index], axis=0))
                
            print("END : " + self.modeldir)

In [3]:
# --- 데이터 로딩 ---
datafile = "/src/data/DataSet/SPECData.npy"

print("load data : ", datafile)
data = np.load(datafile, allow_pickle=True)
print("load complete")

load data :  /src/data/DataSet/SPECData.npy
load complete


In [4]:
# --- 테스트용 데이터와 라벨 사용 ---
testInput = data[0][12000:]
testLabel = data[2][12000:]

In [5]:
# --- 모델별로 분석 수행 ---
modelList = [
    EsvibAnalyzer("/src/hyebin/model/SPEC-01"),
    EsvibAnalyzer("/src/hyebin/model/SPEC-02"),
    EsvibAnalyzer("/src/hyebin/model/SPEC-03")
]
threadList = [threading.Thread(target=modelList[index].analysis, args=(testInput,)) for index in range(len(modelList))]

for thread in threadList:
    thread.setDaemon(True)
    thread.start()
    
for thread in threadList:
    thread.join()

START : /src/hyebin/model/SPEC-03
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
START : /src/hyebin/model/SPEC-02
START : /src/hyebin/model/SPEC-01
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /src/hyebin/model/SPEC-02/variables/variables


Exception in thread Thread-3:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-7f0aa6d3efb4>", line 13, in analysis
    tf.saved_model.loader.load(sess, ["ver1"], self.modeldir)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/util/deprecation.py", line 324, in new_func
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/saved_model/loader_impl.py", line 268, in load
    loader = SavedModelLoader(export_dir)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/saved_model/loader_impl.py", line 284, in __init__
    self._saved_model = parse_saved_model(export_dir)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/saved_model/loader_impl.py", line 83, in parse_saved_model
    constants.SAVED_MODEL_FIL